# Task 2 - Information Retrieval System

## Document Indexing Section

In this section, the content found within the XML raw (and CDATA) tag will be extracted, tokenised and preprocessed (using case-folding, stop-word removal and stemming). Then, extracting all unique index terms (building the vocabulary/bag-of-words/terms of the model). Afterwards, the Document-Term matrix is constructed containing TF-IDF weighting scheme.

### Document Parsing

1. Opening and reading all documents found within the corpus/dataset. 

2. Extract the content found within the XML's raw tag for each of the documents.

3. Parse and preprocess (using NLTK tools) this content and append it to a list (detokenise it before). Therefore, this list will contain the preprocessed content for each parsed XML file.
    

In [1]:
# Importing neccessary modules for this jupyter notebook

from bs4 import BeautifulSoup #Using BeautifulSoup for parsing of XML files
import bs4
from glob import glob

#NLTK Tools - library
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize.treebank import TreebankWordDetokenizer

import string

pth ="docs-raw-texts/" #the path where all XML files reside
files = glob(pth+"*.naf") #getting all the XML files contained in the folder
files.sort() #arranging files in ascending order

content = []
stop_words = set(stopwords.words("english")) #list containing stop words predefined by NLTK
stop_words.add('.')
stop_words.add(',')
stop_words.add("(")
stop_words.add(')')
stop_words.add('"')
stop_words.add("'")

ps = PorterStemmer() #calling PorterStemmer class for words stemming

# Read every XML file
docs = [] # List containing all the parsed content of each document
documents = []
for file in files:

    with open(file, "r", encoding="UTF-8") as file:
        
        # Read each line in the file, readlines() returns a list of lines
        content = file.readlines()
        # Combine the lines in the list into a string
        content = "".join(content)
        bs_content = BeautifulSoup(content, "html.parser") #using html.parser as to not strip the contents in <raw> tag
        
        #Getting the string contained within cdata
        new_bs_content = bs_content.find(text=lambda tag: isinstance(tag, bs4.CData)).string.strip()
        
        tokenisation_words = word_tokenize(new_bs_content)
        
        #Casefolding - converting word to lowercase
        casefolded_sentence = []
        for word in tokenisation_words:
            casefolded_sentence.append(word.casefold())
        
        filtered_sentence = []
        #Stop word removal - removing frequent and non-needed terms in documents
        for word in casefolded_sentence:
            if word not in stop_words:
                filtered_sentence.append(word)
        
        stemming_sentence = []
        #Word stemming - stripping off the stem of each word
        for word in filtered_sentence:
            word = ps.stem(word)
            stemming_sentence.append(word)
            
        #every element in this list represents the preprocessed document
        stemming_sentence = TreebankWordDetokenizer().detokenize(stemming_sentence) #Detokenizing doc
        docs.append("".join(stemming_sentence)) #Combining all the elements in the list into 1
        
        #docs.append(new_bs_content)

print(docs)

['william beaumont human digest william beaumont: physiolog digest imag sourc novemb 21 1785 us-american surgeon william beaumont born becam best known “ father gastric physiolog ” follow research human digest william beaumont born lebanon connecticut becam physician serv surgeon ’ mate armi war 1812 open privat practic plattsburgh new york rejoin armi surgeon 1819 beaumont station fort mackinac mackinac island michigan earli 1820 exist protect interest american fur compani fort becam refug wound 19-year-old french-canadian fur trader name alexi st. martin shotgun went accid american fur compani store close rang june 6th 1822 st. martin ’ wound quit seriou stomach perfor sever rib broken nobodi realli expect young man would surviv realli skin around st. martin ’ wound fuse hole stomach leav perman open – gastric fistula [1] beaumont quickli notic much research potenti back much known digest system order gain inform beaumont perform numer experi st. martin period eight year experi must 

### Extraction of Index Terms

Defining a function which will be used to build the vocabulary/bag-of-words, a.k.a all the terms in all documents (duplicates are not considered).

This function does the following:
    
    1. Loops through each parsed document.
    2. Tokenises the text.
        a. Check if current word is already within the vocabulary.
            i. If True, then increment the value for this word by 1
            ii. Otherwise, set the value for this word to 1.
    3. The vocabulary dictionary is finally returned.

Hence, when this function is called, it returns a dictionary containing all the non-duplicate terms in the corpus together with their frequency throughout all the documents (term: totalTermFrequencyInAllDocs). The terms within this dictionary are its keys whilst the other is its value

In [2]:
# Defining the function used to create the bag-of-words/vocabulary/terms

def buildVocab(docList):
    vocab = {} #dictionary which will hold every term (as keys) and the totalTermFrequencyInAllDocs for each term (as values)
    for doc in docList:
        doc = doc.translate(str.maketrans('', '', string.punctuation))
        
        words = word_tokenize(doc) #tokenising the preprocessed contents
        
        # Looping through all the tokenised words; if word is already in dictionary, then increment its value by 1.
        # Otherwise, Create a new key having 1 as its value
        for word in words:
            if(word in vocab.keys()):
                vocab[word]+= 1
            else:
                vocab[word] = 1
    return vocab

vocab = buildVocab(docs) #calling function
#print(vocab)

### Building the Document-Term Matrix 

To build the Document-Term Matrix: 

    1. Calculate the term frequency (TF) for each of the terms in a document.
    2. Generate the max TF for each of the documents (to be used for term normalisation).
    3. Calculate the IDF for each term in vocabulary.
    4. Build the Document-Term matrix/dataframe containing the TF.IDF weight for each term within each document.
    
It is important to note that TF.IDF is the TF multiplied by its IDF. Moreover, the Document-Term matrix/dataframe will contain documents as rows whilst terms as columns.

#### Calculate Term Frequency

Start off by calculating the term frequency for each term within a document (do the rest for other documents in corpus).

In [3]:
# Importing other needed modules
import numpy as np
import pandas as pd

termDict = {}

docsTFMat = np.zeros((len(docs), len(vocab)))

docsIdfMat = np.zeros((len(vocab), len(docs)))

# Creating a dataframe - Document-Term dataframe. Columns are terms, rows are documents.
docTermDf = pd.DataFrame(docsTFMat, columns=vocab.keys()) 

docCount = 0

#Looping through all the documents, i.e. rows in dataframe.
# During each iteration, tokenising the contents of that respective document.
# Then, if the token is a key in the vocabulary/bag-of-words, then increment its TF by 1.
# Hence, the following code is calculating the TF of each term within that particular document
for doc in docs:
    doc = doc.translate(str.maketrans('', '', string.punctuation))
    words = word_tokenize(doc)
    for word in words:
        if(word in vocab.keys()):
            docTermDf[word][docCount] += 1
    
    docCount+=1 #incrementing counter to iterate to next document
    
#print(docTermDf)

#### Generating a list containing the max TF for each document

In [4]:
max_tf_list = [] # List holding the max TF of each of the documents

#Looping through all the documents, finding the max TF and storing it inside a list
for index in range(len(docs)):
    max_tf = max(docTermDf.iloc[index])
    max_tf_list.append(max_tf)

#### Calculate the IDF for each term

The next step would be that of working out the IDF weights for each term. This will be done by looping through every column in the matrix, and performing the formula for the IDF on each term.

In [5]:
idfDict = {} # Dictionary containing all the IDF values for each document

# Looping through every column/term within the Doc-term dataframe.
# For every column/term, work out its IDF using the formula
# Formula: log10(N/df(t i)) - meaning the log base 10 of the total number of documents in corpus divided by the document frequent of the current term/column
# IMP: Document Frequency (df) is the number of times a particular term appears in the corpus
# DF is calculated by summing up the number of times the TF of that term is > 0 for every document
for column in docTermDf.columns:
    idfDict[column] = np.log10((len(docs))/((docTermDf[column] != 0).sum()))

#print(idfDict)

#### Building the Document-Term Matrix

Now, creating a dataframe which contains the Term-Document Matrix with the TF.IDF weighting scheme.
Initially, the dataframe contains nought values only.

In [8]:
#Document-Term Matrix with TF.IDF weights - Initially set to 0
docsTfIdfMat = np.zeros((len(docs), len(vocab))) 

# Document-Term dataframe with TF.IDF weights
docTfIdfDf = pd.DataFrame(docsTfIdfMat, columns=vocab.keys())

# more options can be specified also
#with pd.option_context('display.max_rows', 100, 'display.max_columns', 100):  
#    print(docTfIdfDf)

After having created the data frame which will hold the Document by Term matrix, it is now time to update the values/weights within this dataframe.

As previously mentioned, TF.IDF weighting scheme is used. Hence, loop through all the rows of the dataframe, get the value of each term and work out its TF-IDF. The Tf.IDF is essentitally multiplying the TermFrequency by its IDF. It is important to note that term frequency normalisation is also done - generate number between 0 and 1 to eliminate bias for longer documents

In [7]:
#Updating each and every value in the Doc-Term dataframe with the corresponding TF.IDF weight of that term

# Looping through each document.
# For each document, loop through all the terms of that document and calculate the respective TF.IDF weight
docCount = 0
for doc in docs:
    for key in idfDict.keys():
        docTfIdfDf[key][docCount] = (docTermDf[key][docCount]/max_tf_list[docCount]) * idfDict[key]
    docCount+=1

# more options can be specified also - displaying more rows & columns of the DF
#with pd.option_context('display.max_rows', 200, 'display.max_columns', 200):  
#    print(docTfIdfDf)

print(docTfIdfDf)

      william  beaumont     human    digest  physiolog      imag     sourc  \
0    0.342024  2.519828  0.124829  1.596537   0.480216  0.058357  0.080294   
1    0.000000  0.000000  0.068089  0.000000   0.000000  0.000000  0.000000   
2    0.000000  0.000000  0.000000  0.000000   0.000000  0.000000  0.000000   
3    0.000000  0.000000  0.000000  0.000000   0.000000  0.053868  0.000000   
4    0.000000  0.000000  0.000000  0.000000   0.000000  0.000000  0.000000   
..        ...       ...       ...       ...        ...       ...       ...   
326  0.000000  0.000000  0.000000  0.000000   0.000000  0.000000  0.000000   
327  0.000000  0.000000  0.000000  0.000000   0.000000  0.000000  0.000000   
328  0.000000  0.000000  0.000000  0.000000   0.000000  0.000000  0.000000   
329  0.273619  0.000000  0.000000  0.000000   0.000000  0.000000  0.000000   
330  0.000000  0.000000  0.000000  0.000000   0.000000  0.000000  0.000000   

       novemb        21      1785  ...   apporov       rio     

## Query Component Section

After successfully constructing the Document-Term matrix, it is now time to get a query, parse it, preprocess it and performing cosine similarity on each of the documents. However, prior to working out the cosine similarities for each of the documents, it is vital to construct the query vector which will contain its own TF.IDF weights and must have the same dimension as the document vector (explained further along the way).

### Getting a Query, Tokenising it and Preprocessing it

Firstly, get a query from one of the files found inside the respective corpus. Then, tokenise it and preprocess (using casefolding, stop word removal and stemming) it.

In [9]:
query = "Fabrication of music instruments" #obtained from wes2015.q01.naf file

#Tokenising the string into a list of words
tokenised_query = word_tokenize(query)
        
#Casefolding 
casefolded_query = []
for word in tokenised_query:
    casefolded_query.append(word.casefold())
        
filtered_query = []
#Stop word removal
for word in casefolded_query:
    if word not in stop_words:
        filtered_query.append(word)
        
stemming_query = []
#Word stemming 
for word in filtered_query:
    word = ps.stem(word)
    stemming_query.append(word)

#Detokenizing query
stemming_query = TreebankWordDetokenizer().detokenize(stemming_query)
print(stemming_query)

fabric music instrument


### Construct the Query Vector

To construct the query vector: 

    1. Calulcate the TF for each query word (usually 1).
    2. All other terms that are not the terms in query, set their TF to 0.
    3. For every term in query, get the IDF of that term, and multiply by its respective TF (same TF.IDF weighting method as before)

Therefore, the query has the same dimension as the document vector but has different weights.

#### Generate Dictionary containing each term in query together with its TF

In [10]:
# Calculate the TF of each term in the query

dictOfQuery = {}
queryWords = word_tokenize(stemming_query) # tokenising query 

# Loop through each of the tokenised query words, and for each word, 
# if word is already in vocabulary, then incerement its value by 1. Otherwise, create a new key for this term and set its value to 1. 
# At the end of this loop, you will end up with a dictionary containing the TF of each term in the query (ex: queryTerm: TF)
for word in queryWords:
    if word in dictOfQuery.keys():
        dictOfQuery[word] += 1
    else:
        dictOfQuery[word] = 1
        
#print(dictOfQuery)

#### Generate Dictionary containing each term in vocabulary together with its respective TF.IDF weight

In [11]:
# Calculating the TF.IDF weights for each term in query. 
# All words not in query will have their weight set to 0.

queryVocab = vocab

#Setting all vocabulary/bag-of-words terms with TF to 0
for key in queryVocab.keys():
    queryVocab[key] = 0

# Looping through the size of the tokenised query, 
# and then looping through all the dictionary keys/terms.
# If the key in vocab dictionary is the same as the term in query, then update its weight to its respective TF.IDF
# IDF of key is retrieved from the dictionary of IDFs previously created when construction Doc-Term matrix
for index in range(len(queryWords)):
    for key in queryVocab.keys():
        if (key == queryWords[index]):
            queryVocab[key] = dictOfQuery[key] * idfDict[key] #calculating TF.IDF
            
print(queryVocab)

{'william': 0, 'beaumont': 0, 'human': 0, 'digest': 0, 'physiolog': 0, 'imag': 0, 'sourc': 0, 'novemb': 0, '21': 0, '1785': 0, 'usamerican': 0, 'surgeon': 0, 'born': 0, 'becam': 0, 'best': 0, 'known': 0, '“': 0, 'father': 0, 'gastric': 0, '”': 0, 'follow': 0, 'research': 0, 'lebanon': 0, 'connecticut': 0, 'physician': 0, 'serv': 0, '’': 0, 'mate': 0, 'armi': 0, 'war': 0, '1812': 0, 'open': 0, 'privat': 0, 'practic': 0, 'plattsburgh': 0, 'new': 0, 'york': 0, 'rejoin': 0, '1819': 0, 'station': 0, 'fort': 0, 'mackinac': 0, 'island': 0, 'michigan': 0, 'earli': 0, '1820': 0, 'exist': 0, 'protect': 0, 'interest': 0, 'american': 0, 'fur': 0, 'compani': 0, 'refug': 0, 'wound': 0, '19yearold': 0, 'frenchcanadian': 0, 'trader': 0, 'name': 0, 'alexi': 0, 'st': 0, 'martin': 0, 'shotgun': 0, 'went': 0, 'accid': 0, 'store': 0, 'close': 0, 'rang': 0, 'june': 0, '6th': 0, '1822': 0, 'quit': 0, 'seriou': 0, 'stomach': 0, 'perfor': 0, 'sever': 0, 'rib': 0, 'broken': 0, 'nobodi': 0, 'realli': 0, 'expect'

#### Calculating Cosine Similarity between Query and each Document

In [24]:
# Till now, we have calculated the TF.IDF weights for each term in our query tokinesed words.
# Now, perform cosine similarity for each document in corpus.

# Start off by working out the numerator of this formula

numeratorSumList = [] # Contains the numerator value of cosine similarity formula for each of the documents

# Loop through each document, and get its row, i.e. all the terms of that document.
# Then, loop through each of the term of that document, 
# and multiply the TF-IDF of that term (in doc-term df) by the TF-IDF of the corresponding term in the query.
# Sum up all these multiplications (formula of cosine similarity - numerator)
# All results are appended to a list

for index in range(len(docs)):
    row = docTfIdfDf.iloc[index] #retrieving each row in df
    summation = 0
    for count in range(len(row)):
        summation += row.iloc[count] * queryVocab[list(queryVocab)[count]] # Calculating numerator
    
    numeratorSumList.append(summation) # append result to list   
    
#print(numeratorSumList)    

In [25]:
# Working out parts of the denominator of cosine similarity function

# importing the math module - for sqaure and square root
import math  

# First, calulcate the magnitude of each of the documents vectors
docMagnitudeSumList = [] # Contains the magnitude value of each document vector - part of denominator of formula

# Loop through every document in corpus, and get its terms. 
# Then, loop through every element/term and sum up the sqaure of the weights of all terms. 
# Finally, append the square root of the result to list.
for index in range(len(docs)):
    row = docTfIdfDf.iloc[index] #retrieving each row in df
    magnitudeOfDoc = 0
    for count in range(len(row)):
        magnitudeOfDoc += math.pow(row.iloc[count], 2) # Sum of squares of weights
    docMagnitudeSumList.append(math.sqrt(magnitudeOfDoc)) # square rooting the sum of squares (of doc vector) before appending
    
#print ("DocMag: ",docMagnitudeSumList)  

#queryMagnitudeSumList = []

# Now, calulcate the magnitude of the query vector - similar to above but only one vector is involved, i.e. query
magnitudeOfQuery = 0
for key in queryVocab.keys():
    magnitudeOfQuery += math.pow(queryVocab[key], 2)  # Sum of squares of weights

magnitudeOfQuery = math.sqrt(magnitudeOfQuery)  # square rooting the sum of squares (of query vector)
#print("QueryMag: ", magnitudeOfQuery)
#queryMagnitudeSumList.append(math.sqrt(magnitudeOfQuery))    

In [26]:
# Working out the cosine similarity function for each of documents with the query

cos_sim = [] # List that will hold the cosine similarity result between the query and each of the documents

# Looping through all the documents, 
# and calculating the cosine similarity between the query and each document using the cosine similarity formula
for index in range(len(docs)):
    numerator = numeratorSumList[index] #Getting respective document numerator
    denominator = docMagnitudeSumList[index] * magnitudeOfQuery # Calulating denominator for that respective document
    similarity = numerator/denominator #calculating cosine similarity between document and query
    cos_sim.append(similarity) # appending result to list
    
#print(cos_sim)

### Output Results as a Ranked List

In this section, output the results of the cosine similarities in descending order - the higher the cosine similairty, the more similar a query vector is with that respective document

In [27]:
#Importing operator module
import operator

# Dictionary will hold all the non-zero cosine similarity values - zero values are discarded since there are completely no features in common
rankedDict = {}
for element in cos_sim:
    if (element > 0):
        #adding items to dictionary; key = index of array, item = cosine similarity
        rankedDict[cos_sim.index(element)] = element 
        
#Sorting the Dictionary in descending order accroding to the values of each key        
rankedDict = dict( sorted(rankedDict.items(), key=operator.itemgetter(1), reverse=True))

# Outputting the ranked list
counter = 0
print("Rank No\t\tDocument No\t\tCosine Similarity")
print("--------------------------------------------------------------")
for x,y in rankedDict.items():
    print(counter+1,"\t\t",x+1,"\t\t\t",y)
    counter+=1



Rank No		Document No		Cosine Similarity
--------------------------------------------------------------
1 		 16 			 0.11691244314568296
2 		 85 			 0.07110608315709632
3 		 259 			 0.0691304316041063
4 		 254 			 0.06738911350148459
5 		 186 			 0.05118557963817511
6 		 209 			 0.04774957996622842
7 		 153 			 0.035216465307171374
8 		 8 			 0.035102019972229816
9 		 170 			 0.03104641894814093
10 		 163 			 0.028965723893595117
11 		 185 			 0.02787781721987226
12 		 215 			 0.027826708818605654
13 		 154 			 0.02539351934997409
14 		 315 			 0.024322028484695876
15 		 296 			 0.02268271233665753
16 		 89 			 0.02267143682325438
17 		 60 			 0.020050255423899065
18 		 4 			 0.018976086688216234
19 		 6 			 0.01891147598685926
20 		 99 			 0.01829289592403888
21 		 162 			 0.017905288905377605
22 		 243 			 0.01610193745256146
23 		 100 			 0.015650590829110935
24 		 179 			 0.014836720825571617
25 		 94 			 0.014834334257087332
26 		 145 			 0.01426067112284951
27 		 39 			 0.014159519

# ----------------------------------- End -----------------------------------